# Review

We are using 
- Anaconda
- JupyterLab
- Julia in Jupyter notebooks
- GitHub

We are using Julia because:
- The syntax is simple, consistent and similar to most procedural languages.  
- Extensions and packages are easily installed through the Julia package manager.
- It provides simple uniform access to a broad range of scientific libraries and examples.
- It is efficiently implemented with minimal overhead i.e. it is performant.

We need to learn how to load packages and write functions containing simple loops in Julia. 
  

# MarkDown and Code Notebook Cells

A Jupyter notebook contains two types of cells. 
- Markdown: For writing text and math
- Computaion: For big and small computations

This is a markdown cell. To edit it simply click on it and type.  Almost all the simple foramating in markdown cells is described in the link below.
-https://www.markdownguide.org/cheat-sheet/
We will come back to Markdown after we have worked out how to perform some simple computations.

# Making Editing, and Deleting Cells
The Edit menu lets you copy and paste cells: keyboard short cuts are beside some of the commands. Some usefull commands are harder to find on the menus. 

- "esc" a inserts a new cell above the cell the cursor is in.
- "esc" b inserts a new cell below the cell the cursor is in.
- "esc" dd deletes the cell the cursor is in.
- "esc" m converts the cell the cursor is in to Markdown
- "esc" y converts the cell the cursor is in to code.

You can display pseudocode fairly simply

# Code Cells and Programming

Most Julia code is a direct translation from other procedural languages. The easiest way to demonstrate this is to write a function that implements a standard linear algebra operation.  We are going to use the Cholesky Decomposition
$$
A = L.L^\top
$$
of a real Symmetric Positive Definite (SPD) matrix A as a product of the Lower Triangular factor $L$ 
and its transpose. The wiki page 
- https://en.wikipedia.org/wiki/Cholesky_decomposition
describes the Cholesky decomposition and provides pseudo code for a variety of slightly different algorithms.    

Reminders:
- A is Symmetric if $A=A^\top$ i.e. $A_{ij}=A_{ji}$
- A is Positive Definite if $x.A.x > 0$ for $x\neq 0$.
- L is lower triangular if $L_{ij}=0$ when $i>j$.

## Wiki Pseudocode
The first pseudo-code on the Wiki page is:
``` lang=C
for (i = 0; i < dimensionSize; i++) {
    for (j = 0; j <= i; j++) {
        float sum = 0;
        for (k = 0; k < j; k++)
            sum += L[i][k] * L[j][k];

        if (i == j)
            L[i][j] = sqrt(A[i][i] - sum);
        else
            L[i][j] = (1.0 / L[j][j] * (A[i][j] - sum));
    }
}
```
This is actually C Code and a few things need to change to to translate it into Julia. We need to 
- Convert the for(start; test; increment) do loops into Julia for-index-in-range-do-end
- Convert the index 0 pseudocode to linear algebra index 1 code
- Convert the psuedocode if(cond) statements into Julia if-else-end code
- Convert the pseudo-code L[i][j] and A[i][j] into the Julia versions L[i,j] and A[i,j] 
- Remove or fix type declerations like float.
We are going to do this slowly.

Note the chances of the partially-translated intermediate versions working in any computer language is vanishingly small. 

## Step 1: Convert to base 1 pseudo code
Changing the base index and writing m for the matrix dimension we get
``` lang=C
for (i = 1; i <= m; i++) {
    for (j = 1; j <= i; j++) {
        float sum = 0;
        for (k = 1; k <= j; k++)
            sum += L[i][k] * L[j][k];
        if (i == j)
            L[i][j] = sqrt(A[i][i] - sum);
        else
            L[i][j] = (1.0 / L[j][j] * (A[i][j] - sum));
    }
}
```

## Step 2:  Change to Julia for loops
Again taking it slowly. I am going to translate the C for loops into Julia
``` lang=Julia
for i in 1:m
    for j in 1:i 
        float sum = 0;
        for k in 1:j
            sum += L[i][k] * L[j][k]
        end
        if (i == j)
            L[i][j] = sqrt(A[i][i] - sum);
        else
            L[i][j] = (1.0 / L[j][j] * (A[i][j] - sum));
    end
end
```

## Step 3:  Change to Julia indexing
I am going to delete the type declaration and translate the matrix indexing into Julia  
``` lang=Julia
for i in 1:m
    for j in 1:i
        sum = 0;
        for k in 1:j
            sum += L[i,k] * L[j,k]
        end
        if (i == j
            L[i,j] = sqrt(A[i,i] - sum);
        else
            L[i,j] = 1.0 / L[j,j] * (A[i,j] - sum));
    end
end
```

## Step 4:  Change to Julia ifs
I am going to translate the if and delete a bunch of semi-colons
``` lang=Julia
for i in 1:m
    for j in 1:i
        sum = 0.0
        for k in 1:j
            sum += L[i,k] * L[j,k]
        end
        if i == j
            L[i,j] = sqrt(A[i,i] - sum)
        else
            L[i,j] = 1.0 / L[j,j] * (A[i,j] - sum)
        end
    end
end
```
The result is code that will run in Julia and produce the Cholesky factor L of an SPD A. Here is the resulting Julia script and a simple test. 

In [34]:
using LinearAlgebra
m=12
A=rand(m,m); A=A*A'
L=zeros(m,m)

for i in 1:m
    for j in 1:i 
        sum = 0.0
        for k in 1:j
            sum += L[i,k] * L[j,k]
        end
        if i == j
            L[i,j] = sqrt(A[i,i] - sum)
        else
            L[i,j] = 1.0 / L[j,j] * (A[i,j] - sum)
        end
    end
end
display(norm(A-L*L'))
display(istril(L))
L

8.881784197001252e-16

true

12×12 Array{Float64,2}:
 2.01964  0.0        0.0        0.0        …   0.0       0.0        0.0     
 1.77756  1.34777    0.0        0.0            0.0       0.0        0.0     
 1.89891  0.581049   0.939914   0.0            0.0       0.0        0.0     
 1.59368  0.468795   0.0874578  1.17544        0.0       0.0        0.0     
 1.91655  0.832954  -0.391617   0.505465       0.0       0.0        0.0     
 1.5161   0.568871   0.442217   0.0973646  …   0.0       0.0        0.0     
 1.61055  0.293268  -0.12836    0.363748       0.0       0.0        0.0     
 1.71965  0.226814   0.140693   0.454195       0.0       0.0        0.0     
 1.01144  0.532409   0.154745   0.441326       0.0       0.0        0.0     
 1.71085  0.303734  -0.309798   0.355805       0.470677  0.0        0.0     
 1.49718  0.397585   0.234631   0.40999    …   0.130214  0.315462   0.0     
 1.37179  0.246799  -0.381377   0.186927      -0.483312  0.0190166  0.261785

# Loops and BLAS

Basic linear algebra (and some not so basic linear algebra) are implemented very efficiently on every computer. The basic linear algebra is contained in sub routines called Basic Linear Algebra Subroutines (BLAS) tuned for specific hardware. Some history and details are on wikipedia at
- https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms
What this means is code will run faster if we replace any loop that does basic linear algebra with the appropriate linear algebra command.  The inner most loop simply accumulates an inner product in the variable sum.  Replacing
``` lang=Julia
    sum = 0.0
        for k in 1:j
            sum += L[i,k] * L[j,k]
        end
```
with the easier to read BLAS version
``` lang=Julia
    sum = dot(L[i,1:j],L[j,1:j])
```
gives cleaner code that runs faster! 

In [35]:
using LinearAlgebra
m=12
A=rand(m,m); A=A*A'
L=zeros(m,m)

for i in 1:m
    for j in 1:i 
        sum = dot(L[i,1:j],L[j,1:j])
        if i == j
            L[i,j] = sqrt(A[i,i] - sum)
        else
            L[i,j] = 1.0 / L[j,j] * (A[i,j] - sum)
        end
    end
end
display(norm(A-L*L'))
display(istril(L))
L

1.4043333874306805e-15

true

12×12 Array{Float64,2}:
 2.05567   0.0         0.0        …   0.0        0.0       0.0     
 2.12096   1.07325     0.0            0.0        0.0       0.0     
 1.74996   0.115732    1.24133        0.0        0.0       0.0     
 1.1405    0.706499    0.321516       0.0        0.0       0.0     
 1.78099   0.33436     1.02258        0.0        0.0       0.0     
 1.89779   0.0981113   0.424826   …   0.0        0.0       0.0     
 0.987083  0.56677     0.904847       0.0        0.0       0.0     
 1.42519   0.397774   -0.0733879      0.0        0.0       0.0     
 1.16721   0.27522     1.10508        0.0        0.0       0.0     
 1.61103   0.472963    1.13947        0.600308   0.0       0.0     
 1.99487   0.175406    0.504603   …   0.338797   0.36459   0.0     
 1.79247   0.953523    0.4592        -0.0589443  0.217713  0.400327

# Functions and Timing

It is easy to make functions out of our two working scripts and compare timings

In [3]:
using LinearAlgebra
function CholeskyWiki(A) 
m = size(A)[1]
L = zeros(m,m)
for i in 1:m
    for j in 1:i 
        sum = dot(L[i,1:j],L[j,1:j])
        if i == j
            L[i,j] = sqrt(A[i,i] - sum)
        else
            L[i,j] = 1.0 / L[j,j] * (A[i,j] - sum)
        end
    end
end
L
end

function CholeskyWikiBLAS(A)
m = size(A)[1]
L = zeros(m,m)
for i in 1:m
    for j in 1:i 
        sum = dot(L[i,1:j],L[j,1:j])
        if i == j
            L[i,j] = sqrt(A[i,i] - sum)
        else
            L[i,j] = 1.0 / L[j,j] * (A[i,j] - sum)
        end
    end
end
L
end

CholeskyWikiBLAS (generic function with 1 method)

In [7]:
m = 5*10^2;
A = rand(m,m); A=A*A';
@time LW = CholeskyWiki(A) 
@time LB = CholeskyWikiBLAS(A) 
map(norm,[A-LW*LW',A-LB*LB'])

  0.084738 seconds (250.51 k allocations: 354.049 MiB, 21.50% gc time)
  0.059271 seconds (250.51 k allocations: 354.049 MiB, 24.12% gc time)


2-element Array{Float64,1}:
 3.243554043235799e-11
 3.243554043235799e-11